# 01 – Data Preprocessing

### 📌 Objective  

 This notebook prepares the raw dataset for machine learning by cleaning, transforming, and structuring the data for downstream modeling.

---

### 🧱 Tasks Covered
- Loading raw data
- Exploring data types, missing values, and distributions
- Handling missing or inconsistent values
- Encoding categorical variables
- Scaling numerical features
- Saving the clean dataset for the next stage

---

### 📂 Input
- `student_depression_dataset.csv` saved in `Data/raw/`

### 📦 Output
- `clean_data.csv` saved in `Data/processed/FC110552_mithula-cbw/`

---


 ### 📁 Dataset Characteristics:
- **Data Format:** CSV (each row contains information about one student) - **Identifier:** Student ID (unique for each individual)
- **Demographic Information:** Age, Gender, City of Residence
- **Academic Information:** Cumulative Grade Point Average (CGPA), Academic Pressure, Study Method
- **Lifestyle Factors:** Sleep duration, eating habits, working hours, job satisfaction
- **Psychological and Social Factors:** Family history of mental illness, financial stress, history of suicidal thoughts

### Target Variable:
- Depression_Status: A binary indicator showing whether a student experiences depression (0/1 or Yes/No)

---

In [3]:
# importing necessary libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Standard Libraries
import warnings

In [9]:
# Assign the CSV file to a Pandas DataFrame.
df = pd.read_csv("./Data/raw/student_depression_dataset.csv")